<a href="https://colab.research.google.com/github/Videothek/machine-learning/blob/main/RAG_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation benötigter Pakete

In [1]:
# Installieren der notwendigen Pakete.
!pip install PyMuPDF pytesseract Pillow langchain-community langchain_openai docarray
!apt-get update
!apt-get install tesseract-ocr

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

## Definieren relevanter Variablen

In [2]:
# Name des Google-Colab-Secret für den OpenAI-Key
colab_secret = "oai_apikey"

# Festlegen der URL von der das PDF-Dokument geladen werden kann.
url = "https://raw.githubusercontent.com/Videothek/machine-learning/main/MCP.pdf"

# Embedding Model für das Chunking festlegen.
embedding_model="text-embedding-3-small"

# Eingabe der Chunk Größe (Token/Chunk)
input_chunk_size = 300

# Eingabe des Overlaps (Festlegen der Informationslücke)
input_overlap = 50

# Large Language Model für die Generierung der Antwort festlegen.
large_language_model = "gpt-4o-mini"

# Eingabe der maximalen Output-Token des LLM
max_output_tokens = 300

In [3]:
# Importieren des Pakets für den Zugriff auf den Google-Colab-KeyStore.
from google.colab import userdata

# Auslesen des OpenAI-Key aus dem Google-Colab-KeyStore.
OPENAI_API_KEY = userdata.get(colab_secret)

## Vorbereiten des PDF-Dokument

In [4]:
# Importieren der notwendigen Pakete für den Abruf des PDF-Dokument.
import requests
import io

# Herunterladen der pdf Datei als Kontext für das RAG.
response = requests.get(url)

# Speichern des PDF-Byte-Stream.
pdf_file = io.BytesIO(response.content)

print(f"Status Code: {response.status_code}")

Status Code: 200


In [5]:
# Importieren von PyMuPDF um das PDF in Bilder zu verwandeln.
import fitz

# Array für die PDF-Bilder initialisieren.
pdf_images = []

# PDF-Datei öffnen.
doc = fitz.open(stream=pdf_file, filetype="pdf")

# Über die Seiten des PDF-Dokument iterieren.
for page_num in range(len(doc)):

    # Seite des PDF-Dokument laden.
    page = doc.load_page(page_num)

    # PDF-Dokument als pixmap laden.
    pix = page.get_pixmap(dpi=300)

    # Pixmap in PNG konvertieren.
    img_bytes = pix.tobytes(output="png")

    # Seite des PDF-Dokument zum Array hinzufügen.
    pdf_images.append(img_bytes)

# PDF-Dokument schließen.
doc.close()

print(f"Seiten des PDF-Dokumentes: {len(pdf_images)}")

Seiten des PDF-Dokumentes: 8


In [6]:
# Importieren der Pakete um Text aus den PDF-Bildern auszulesen.
import pytesseract
from PIL import Image

# Array für Textinhalte des PDF-Dokument initialisieren.
extracted_text = []

# Durch die Bilder des PDF-Dokument iterrieren.
for image_bytes in pdf_images:

    # Bild des PDF-Dokument öffnen.
    image = Image.open(io.BytesIO(image_bytes))

    # Text aus dem Bild extrahieren.
    text = pytesseract.image_to_string(image)

    # Text aus dem Bild zum Array hinzufügen.
    extracted_text.append(text)

print(f"Seiten mit erkanntem Text: {len(extracted_text)}")

Seiten mit erkanntem Text: 8


In [7]:
# Variable für den Text definieren.
pdf_document = ""

# Durch den erkannten Text iterrieren.
for text in extracted_text:

    # Text zusammenfügen.
    pdf_document += text

print(f"Anzahl der erkannten Zeichen: {len(pdf_document)}")

Anzahl der erkannten Zeichen: 8178


## Implementierung des RAG-Systems

In [8]:
# Importieren des Pakets um den Text aus dem PDF-Dokument in Chunks zu zerlegen.
import tiktoken

# Encoding für das genutzte Embedding-Modell abrufen,
# dies hilft die Tokenanzahl zu kontrollieren und Kosten zu optimieren.
encoding = tiktoken.encoding_for_model(embedding_model)

# Das Dokument wird in Tokens zerlegt, welche die kleinste Einheit sind, die das LLM verarbeitet.
# Das zerlegen in Token folgt dem speziellen Encoding für das jeweilige Embedding-Modell.
# Die Token werden in diesem Fall benötigt, um das PDF-Dokument in Chunks zu zerlegen
tokens = encoding.encode(pdf_document)

# Diese Variable gibt an, wie viele Token maximal in jedem Chunk des PDF-Dokument sein dürfen.
chunk_size = input_chunk_size

# Diese Variable gibt an, wie viele Token in einem Chunk überlappen, um den Kontext besser zu erhalten.
# Dadurch werden Informationslücken am Chunkrand vermieden.
overlap = input_overlap

# Array für die Chunks initialisieren.
chunks = []

# Kontrollvariable für die while-Schleife initialisieren.
start = 0

# Alle erstellten Tokens durchlaufen, bis alle Token einem Chunk zugeordnet wurden.
while start < len(tokens):

    # Speichern der für die chunk_size festgelegten Anzahl an Token in einen Chunk.
    chunk = tokens[start:start + chunk_size]

    # Den neuen Chunk zum Array aller Chunks hinzufügen.
    chunks.append(encoding.decode(chunk))

    # Bei den Token um die festgelegte overlap Anzahl zurückgehen, um den overlap zu berücksichtigen
    # und Informationsverlust zu vermeiden.
    start += chunk_size - overlap

print(f"Anzahl der Token: {len(tokens)}")
print(f"Anzahl der Chunks: {len(chunks)}")

Anzahl der Token: 2356
Anzahl der Chunks: 10


In [9]:
# Importieren der Pakete um das Embedding-Modell aufzurufen, die Chunks-Liste in eine Documents-Liste zu verwandeln
# und die Embeddings in einer Vektordatenbank zu speichern.
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS

# Festlegen des Embedding-Modells, für die spätere Nutzung bei dem Embedding der Chunks.
embeddings = OpenAIEmbeddings(

    # Festlegen des Embedding-Modells.
    model=embedding_model,

    # Festlegen des API_KEY für OpenAI.
    openai_api_key=OPENAI_API_KEY
)

# Verwandeln der Chunks-Liste in eine Documents-Liste, um diese als einen FAISS-Vektor speichern zu können.
chunks_document = [Document(page_content=chunk) for chunk in chunks]

# Embedding der Chunks und Speicherung in einem FAISS-Index für die Nutzung in der Chain.
# In diesem Schritt werden die Chunks in das Embedding-Modell gegeben, welches die Chunks bzw. die Token
# in Word-embeddings verwandelt, um diese für das LLM abrufbar zu machen.
# FAISS steht für Facebook AI Similarity Search und ist eine Open-Source Bibliothek für die Vektorsuche,
# es berechnet später in der Chain die Ähnlichkeit der Frage zu den gespeicherten Word-Embeddings der Chunks
# und gibt die relevantesten Text-Passagen an das LLM weiter, welches basierend darauf die Antwort generiert.
vectorstore = FAISS.from_documents(chunks_document, embeddings)

In [10]:
# Importieren der Pakete um das LLM abzurufen, den Antwort-String aus der OpenAI-Antwort zu extrahieren
# und das Chat-Template zu bauen.
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

# Festlegen des LLM für die Generierung der Antwort in der Chain basierend auf dem Kontext.
model = ChatOpenAI(

    # Festlegen des Large-Language-Modells.
    model=large_language_model,

    # Festlegen des API_KEY für OpenAI.
    openai_api_key=OPENAI_API_KEY,

    # Maximale Anzahl der Ausgabetoken des Large Language Model, um die Kosten unter Kontrolle zu behalten.
    max_tokens=max_output_tokens
)

# Festlegen des Parsers, um die Antwort als String aus dem komplexen Objekt auszulesen,
# welches durch das LLM bzw. OpenAI ausgegeben wird.
# Hierbei handelt es sich um den Standardparser für Langchain chains und kann beispielsweise
# durch einen JsonParser ausgetauscht werden, um JSON-Strukturierte Ausgaben zu erhalten.
parser = StrOutputParser()

# Festlegen des Template, welches genutzt wird, um den Kontext und die Frage an das LLM zu geben.
# Es vereinfacht die Mitgabe von Anweisungen an das LLM, wie es zum Beispiel zu antworten hat,
# wenn die Informationen nicht durch das Dokument beantwortet werden können.
# {context} und {question} sind Platzhalter für die in der chain definierte Paramter.
template = """
Beantworte die Frage basierend auf dem Kontext.
Wenn Du die Frage nicht beantworten kannst, antworte "Ich weiß es nicht".

Context: {context}

Question: {question}
"""

# Vorbereitung eines LangChain Prompt-Objekts um dieses später mit dem Kontext und der Frage
# füllen zu können.
prompt = ChatPromptTemplate.from_template(template)

In [11]:
# Importieren des Pakets um die Eingabe der Frage direkt an die Chain, und damit auch den Prompt, weiterzugeben.
from langchain_core.runnables import RunnablePassthrough

# Definieren der Chain, welche dem Model den zur Frage passenden Kontext aus der FAISS Verktordatenbank,
# sowie die Frage mitgibt, die als Eingabe direkt weitergelitet wird, mitgibt.
# Zudem wird der finale prompt gebaut, indem der Kontext und die Frage in das zuvor gebaute Template eingefügt wird.
# Anschließend wird das definierte LLM aufgerufen, welches die prompt erhält
# und die Frage anhand des Kontext beantworten soll.
# Anschließend wird die Ausgabe des LLM durch den Parser umgewandelt,
# um den reinen Antwort-String zu erhalten.
# Durch diese Formatierung und Vorgehen, wird die Komplexität einer solchen AI-Chain zusammengefasst und
# übersichtlich dargestellt.
chain = (
    {"context": vectorstore.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)

# Letztendlich wird die zuvor definierte Chain ausgeführt. Die Frage wird dabei, wie zuvor beschrieben,
# direkt in das Template eingebaut.
# Der Retriever durchsucht anschließend die Vektordatenbank nach passenden Textpassagen und gibt
# diese an das LLM weiter, welche basierend auf der Frage und den Textpassagen eine Antwort generiert.
# Anschließend wird die Antwort durch den Parser aus dem zurückgelieferten Objekt gefiltert und ausgegeben.
chain.invoke("Was ist wichtig um effektiv mit MCP arbeiten zu können?")

'Um effektiv mit MCP arbeiten zu können, ist es wichtig, die grundlegenden Konzepte und die damit verbundene Terminologie zu verstehen.'